<a href="https://colab.research.google.com/github/Karincheong/Call-For-Help-American-Sign-Language-Recognition-Using-Deep-Learning-Model/blob/main/CNN_model_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf


In [ ]:
import os
from PIL import Image
import numpy as np

image_dir = '/content/drive/MyDrive/dataset3'
image_files = os.listdir(image_dir)

In [ ]:
import os
import numpy as np

from keras.utils import load_img, img_to_array
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import load_model
import pickle
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from keras import models
import numpy as np
from glob import glob
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import keras
import pathlib
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import models
import pathlib
from tensorflow.keras import initializers
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions

In [ ]:
import numpy as np
import os
import random

# Set path to image directory
data_directory = '/content/drive/MyDrive/dataset3'
# List out subfolders
subfolders = sorted(os.listdir(data_directory))
# Lstore image and labels
image_paths = []
labels = []

# Iterate over subfolders
for subfolder in subfolders:
    folder_path = os.path.join(data_directory, subfolder)
    file_names = os.listdir(folder_path)
    file_paths = [os.path.join(folder_path, file_name) for file_name in file_names]
    image_paths.extend(file_paths)
    labels.extend([subfolder] * len(file_paths))

# Convert to numpy arrays
X = np.array(image_paths)
y = np.array(labels)

# Shuffle the dataset indices
indices = np.arange(len(X))
np.random.shuffle(indices)

# Shuffle the X and y arrays based on the shuffled indices
X = X[indices]
y = y[indices]


In [ ]:
#encode
le = LabelEncoder()
y = le.fit_transform(y)
y = np_utils.to_categorical(y)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7, random_state=42)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [ ]:
trainsize=len(X_train)
trainsize

4889

In [ ]:
testsize=len(X_test)
testsize

1048

In [ ]:
valsize=len(X_val)
valsize

NameError: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np

# Set image dimensions
image_width = 64
image_height = 64
batch_size=20

datagen = ImageDataGenerator(rescale=1.0/255)  # Rescale pixel values between 0 and 1

# Function to load and preprocess images
def load_and_preprocess_images(image_paths):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=(image_width, image_height))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

# preprocess train data
X_train_processed = load_and_preprocess_images(X_train)
y_train = np.array(y_train)

# preprocess validation data
X_val_processed = load_and_preprocess_images(X_val)
y_val = np.array(y_val)

# preprocess test data
X_test_processed = load_and_preprocess_images(X_test)
y_test = np.array(y_test)




In [ ]:
train_generator = datagen.flow(X_train_processed, y_train, batch_size=batch_size)

validation_generator = datagen.flow(X_val_processed, y_val, batch_size=batch_size)

test_generator = datagen.flow(X_test_processed, y_test, batch_size=batch_size)

In [ ]:
# traditional CNN without dropout

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3),dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu',dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu',dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu',dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))
# Displaying a summary of the model
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 conv2d_21 (Conv2D)          (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 conv2d_23 (Conv2D)          (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                 

In [ ]:
# set optimazation
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [ ]:
history = model.fit_generator(
 train_generator,
 steps_per_epoch=100,
 epochs=20,
 validation_data=validation_generator,
 validation_steps=50)

Epoch 1/20


<ipython-input-24-c9aaf91eea1c>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


100/100 [==============================] - 3s 11ms/step - loss: 2.3649 - accuracy: 0.1350 - val_loss: 2.0075 - val_accuracy: 0.3030
Epoch 2/20
100/100 [==============================] - 1s 9ms/step - loss: 1.8062 - accuracy: 0.3585 - val_loss: 1.5946 - val_accuracy: 0.4670
Epoch 3/20
100/100 [==============================] - 1s 9ms/step - loss: 1.2280 - accuracy: 0.5887 - val_loss: 1.2865 - val_accuracy: 0.5870
Epoch 4/20
100/100 [==============================] - 1s 9ms/step - loss: 0.7934 - accuracy: 0.7491 - val_loss: 0.8701 - val_accuracy: 0.7370
Epoch 5/20
100/100 [==============================] - 1s 9ms/step - loss: 0.5025 - accuracy: 0.8517 - val_loss: 0.7264 - val_accuracy: 0.7730
Epoch 6/20
100/100 [==============================] - 1s 9ms/step - loss: 0.3477 - accuracy: 0.9010 - val_loss: 0.5892 - val_accuracy: 0.8320
Epoch 7/20
100/100 [==============================] - 1s 9ms/step - loss: 0.2429 - accuracy: 0.9291 - val_loss: 0.5931 - val_accuracy: 0.8400
Epoch 8/20
100/1

In [ ]:
# Evaluate the model by using test generator
test_loss, test_accuracy = model.evaluate(test_generator)

53/53 [==============================] - 0s 4ms/step - loss: 0.6599 - accuracy: 0.8855


In [ ]:
# traditional CNN with dropout

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3),dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu',dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu',dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu',dilation_rate=(1,1),kernel_initializer='glorot_uniform',strides=(1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))
# Displaying a summary of the model
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 conv2d_25 (Conv2D)          (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 conv2d_27 (Conv2D)          (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                 

In [ ]:
# set optimazation
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [ ]:
history = model.fit_generator(
 train_generator,
 steps_per_epoch=100,
 epochs=100,
 validation_data=validation_generator,
 validation_steps=50)

Epoch 1/100


<ipython-input-34-ba2b11a3c3df>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


100/100 [==============================] - 3s 11ms/step - loss: 2.2013 - accuracy: 0.1415 - val_loss: 2.2323 - val_accuracy: 0.1180
Epoch 2/100
100/100 [==============================] - 1s 9ms/step - loss: 1.9767 - accuracy: 0.2725 - val_loss: 1.7602 - val_accuracy: 0.4070
Epoch 3/100
100/100 [==============================] - 1s 11ms/step - loss: 1.5183 - accuracy: 0.4721 - val_loss: 1.2757 - val_accuracy: 0.6030
Epoch 4/100
100/100 [==============================] - 1s 10ms/step - loss: 1.1779 - accuracy: 0.6130 - val_loss: 1.0509 - val_accuracy: 0.6650
Epoch 5/100
100/100 [==============================] - 1s 10ms/step - loss: 0.8642 - accuracy: 0.7120 - val_loss: 0.7784 - val_accuracy: 0.7770
Epoch 6/100
100/100 [==============================] - 1s 9ms/step - loss: 0.6980 - accuracy: 0.7810 - val_loss: 0.7140 - val_accuracy: 0.7640
Epoch 7/100
100/100 [==============================] - 1s 9ms/step - loss: 0.5735 - accuracy: 0.8100 - val_loss: 0.5890 - val_accuracy: 0.8190
Epoch 8

In [ ]:
# Evaluate the model using the test generator
test_loss, test_accuracy = model.evaluate(test_generator)

53/53 [==============================] - 0s 3ms/step - loss: 0.9747 - accuracy: 0.9065


In [ ]:
# Save the model in HDF5 format
model.save('CNN_callforhelp_ASL.h5')

In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

# Preprocess the input image
image_path = '/content/drive/MyDrive/dataset3/help/bhelp_0_0.jpg'
image = Image.open(image_path)
image = image.resize((64, 64))
image_array = np.array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Load the saved model
model = load_model('CNN_callforhelp_ASL.h5')

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)

# Print the predicted class
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 75ms/step
Predicted class: 0


In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

# Preprocess the input image
image_path = '/content/drive/MyDrive/Capture1.JPG'
image = Image.open(image_path)
image = image.resize((64, 64))
image_array = np.array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Load the saved model
model = load_model('CNN_callforhelp_ASL.h5')

# Perform inference on the input image
predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)

# Print the predicted class
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 96ms/step
Predicted class: 5
